In [1]:
import numpy as np
import matplotlib as mp

import matplotlib.pyplot as plt
import pandas as pd
import import_ipynb

from sklearn import linear_model
from sklearn import cluster
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import MultiLabelBinarizer
from DataPreparation import *

from math import ceil

#functie ajutatoare pentru encodarea datelor
#Returneaza saptamana in care a fost data rezultatului

def encodingTime(df, src):
    for i in range(len(df)): 
        d = int(df.loc[i, 'data rezultat testare'].split('-')[0])
        m = int(df.loc[i, 'data rezultat testare'].split('-')[1].strip("0"))
        df.loc[i, src]= ceil((d + 30.5 * (m - 3)) / 7)    
    
    return df

#Returneaza categoria de varsta in care se afla pacientul
def encodingAge(df, src):
    for i in range(len(df)): 
        if ( df.loc[i,'vârstă'] < 20):
            df.loc[i,src] = '0-20'
        elif ( df.loc[i,'vârstă'] < 40):
            df.loc[i,src] = '20-40'
        elif ( df.loc[i,'vârstă'] < 60):
            df.loc[i,src] = '40-60'
        elif ( df.loc[i,'vârstă'] < 80):
            df.loc[i,src] = '60-80'
        else:
            df.loc[i,src] = '80+'

    return df

#Returneaza lista de simptome de interes
def encodingSimptome(df, src, simptome):

    listSimptomeGlobal = []
    for i in range(len(df['simptome declarate'])):
        x = str(df.loc[i,'simptome declarate']).replace(",", " ").split(" ")  
        listSimptome = []
        for simptom in simptome:   
            if simptom in x or simptom.upper() in x:
                listSimptome.append(simptom)
            else:
                continue
        if listSimptome == []:
            listSimptome.append('nu')

        listSimptomeGlobal.append(listSimptome)

    df_ex = pd.DataFrame({'grup simptome': listSimptomeGlobal})
    df = df.join(df_ex, lsuffix='_caller')

    return df

#returneaza daca a fost suspect de covid sau nu
def encodingDiagnostic(df,src):
    for i in range(len(df['diagnostic și semne de internare'])):
        x = str(df.loc[i,'diagnostic și semne de internare']).lower()
        if 'susp' in x:
            df.loc[i, src] = 'DA'
        else:
            df.loc[i, src] = 'NU'
    
    return df


# Functia de encodare a datelor
def encodingData(df):

    simptome =[ "febra", "tuse", "temp" , "frisoane",
    "seaca", "musculare", "diaree", "dispnee", "disfagee", "durere"]

    label = LabelEncoder()

    #prelucrarea datelor pentru encoding

    df = encodingAge(df, 'grup varsta');
    df = encodingTime(df, 'grup dată rezultat')
    df = encodingTime(df, 'grup dată debut simptome declarate')
    df = encodingTime(df, 'grup dată internare')
    df = encodingDiagnostic(df, 'grup diagnostic')
    df = encodingSimptome(df, 'grup simptome', simptome)

    #LABEL ENCODING
    df['confirmare contact cu o persoană infectată']= label.fit_transform(df['confirmare contact cu o persoană infectată']) 
    df['mijloace de transport folosite']= label.fit_transform(df['mijloace de transport folosite'])
    df['istoric de călătorie']= label.fit_transform(df['istoric de călătorie'])

    df['grup dată debut simptome declarate'] = label.fit_transform(df['grup dată debut simptome declarate'])
    df['dată dată internare'] = label.fit_transform(df['grup dată internare'])
    df['grup dată rezultat'] = label.fit_transform(df['grup dată rezultat'])
    df['grup varsta'] = label.fit_transform(df['grup varsta'])
    df['grup diagnostic'] = label.fit_transform(df['grup diagnostic'])
    df['rezultat'] = label.fit_transform(df['rezultat testare'])
    
    
    #ONE HOT ENCODING
    simptome.append('nu')
    simptome.sort()
    one_hot = MultiLabelBinarizer()
    one_hot.fit([simptome])
    fill_array = [0] * len(df)
    for simptom in simptome:
        df[simptom] = fill_array
    df[simptome] = one_hot.transform((i for i in df['grup simptome']))



    return df




importing Jupyter notebook from DataPreparation.ipynb


In [ ]:
#test
df = prelucrareDate("Date test.csv")
df = encodingData(df)
df

,instituția sursă,sex,vârstă,dată debut simptome declarate,simptome declarate,dată internare,simptome raportate la internare,diagnostic și semne de internare,istoric de călătorie,mijloace de transport folosite,...,disfagee,dispnee,durere,febra,frisoane,musculare,nu,seaca,temp,tuse
0,X,FEMININ,52,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
1,X,FEMININ,40,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
2,X,MASCULIN,69,2020-04-14 00:00:00,ASIMPTOMATIC,2020-04-14 00:00:00,"DURERE LA NIVELUL MEMBRELOR, DISPNEE LA EFFORT",HIDRONEFROZA STG.LITIAZICA PE RINICHI UNIC FUN...,1,1,...,0,0,0,0,0,0,1,0,0,0
3,X,MASCULIN,69,2020-04-07 00:00:00,"astenie,febra",2020-04-13 00:00:00,"astenie,febra","susp.Covid 19 ,pneumonie bilaterala",1,1,...,0,0,0,1,0,0,0,0,0,0
4,X,FEMININ,48,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1284,X,MASCULIN,9,2020-04-15 00:00:00,ARSURA GR. III,2020-04-15 00:00:00,ARSURA,T29.0,1,1,...,0,0,0,0,0,0,1,0,0,0
1285,y,MASCULIN,84,2020-05-08 00:00:00,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
1286,y,MASCULIN,55,2020-05-18 00:00:00,"EDEME, FATIGABILITATE",2020-05-15 00:00:00,"EDEME ,FATIGABILITATE",SUSPICIUNE INFECTIE SARS-COV20. CMD; ICC DECOM...,1,1,...,0,0,0,0,0,0,1,0,0,0
1287,Z,FEMININ,30,2020-05-20 00:00:00,ASIMPTOMATICA,2020-05-20 00:00:00,ASIMPTOMATICA,"G I P I SARCINĂ 40 SĂPT. FĂT VIU, UNIC .PREZE...",1,1,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
df = prelucrareDate("Date antrenare.csv")
df = encodingData(df)
df

,instituția sursă,sex,vârstă,dată debut simptome declarate,simptome declarate,dată internare,simptome raportate la internare,diagnostic și semne de internare,istoric de călătorie,mijloace de transport folosite,...,disfagee,dispnee,durere,febra,frisoane,musculare,nu,seaca,temp,tuse
0,Z,MASCULIN,45,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
1,X,FEMININ,81,2020-04-08 00:00:00,DURERI ABDOMINALE,2020-04-08 00:00:00,DUREI ABD,HEMORAGIE DIGESTIVA,1,1,...,0,0,0,0,0,0,1,0,0,0
2,y,FEMININ,79,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
3,X,FEMININ,71,2020-04-11 00:00:00,"tuse seaca ,febra",2020-04-14 00:00:00,"tuse seaca ,febra",susp.Covid 19,1,1,...,0,0,0,1,0,0,0,1,0,1
4,X,MASCULIN,52,2020-04-14 00:00:00,nu are,2020-04-14 00:00:00,nu are,Retard mental cu tulburari de comportament severe,1,1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3860,y,FEMININ,47,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
3861,y,MASCULIN,73,NaN,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
3862,X,FEMININ,80,2020-04-03 00:00:00,"febra, dificultati respiratorii",2020-04-03 00:00:00,dispnee febra tuse seaca,pneumopatie acuta,1,1,...,0,0,0,1,0,0,0,0,0,0
3863,y,FEMININ,47,2020-04-07 00:00:00,NaN,NaN,NaN,NaN,1,1,...,0,0,0,0,0,0,1,0,0,0
